# P1

In [7]:
import numpy as np
import keras
import time
import os
import cv2
import math
from keras import backend as k
from keras.models import Sequential
from keras.layers import Activation
#from keras.layers import Dense, Flatten, Dropout #type de couches
from keras.layers import Dense, Flatten, Dropout #type de couches
from keras.layers import Conv2D, MaxPooling2D
#from keras.layers import BatchNormalization
from keras.layers import BatchNormalization
#from tensorflow.keras.optimizers import Adam
from keras.optimizers import Adam
from keras.metrics import binary_crossentropy
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from matplotlib import pyplot as plt

ModuleNotFoundError: No module named 'numpy'

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


# P2

In [ ]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_path = 'data/train/'
valid_path = 'data/valid/'

batch_size = 32

# Création des générateurs avec augmentation de données pour l'entraînement

train_datagen = ImageDataGenerator(
    rescale=1.0 / 255,        # Normalisation
    rotation_range=20,        # Rotation aléatoire
    width_shift_range=0.2,    # Décalage horizontal
    height_shift_range=0.2,   # Décalage vertical
    shear_range=0.2,          # Transformation en cisaillement
    zoom_range=0.2,           # Zoom aléatoire
    horizontal_flip=True      # Inversion horizontale
)


# Générateur de validation avec uniquement une normalisation
valid_datagen = ImageDataGenerator(
    rescale=1.0/255.0  # Normalisation sans augmentation de données
)

# Chargement des images augmentées pour l'entraînement
train_batches = train_datagen.flow_from_directory(
    train_path,
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='binary'  # Binaire : car / not_car
)

# Chargement des images de validation
valid_batches = valid_datagen.flow_from_directory(
    valid_path,
    target_size=(100, 100),
    batch_size=batch_size,
    class_mode='binary'
)

# Vérification des indices de classe
print(train_batches.class_indices)  # Devrait afficher {'car': 0, 'not_car': 1} ou l'inverse


In [ ]:
steps_train = train_batches.samples // train_batches.batch_size

In [ ]:
for batch in train_batches:
    print(batch[0].shape)  # Affichez la taille des images dans le batch
    break  # Pour afficher seulement un batch


In [ ]:
steps_valid = valid_batches.samples // valid_batches.batch_size

In [ ]:
num_classes = train_batches.num_classes
num_classes

In [ ]:
cls_train = train_batches.classes
cls_test =  valid_batches.classes
class_names = list(train_batches.class_indices.keys())
class_names

In [ ]:
print(f"Nombre total d'images dans le train set : {train_batches.samples}")
print(f"Nombre total d'images dans le valid set : {valid_batches.samples}")


In [ ]:
def path_join(dirname, filenames):
    return [os.path.join(dirname, filename) for filename in filenames]
image_paths_train = path_join(train_path, train_batches.filenames)
image_paths_test = path_join(valid_path, valid_batches.filenames)

def plot_images(images, cls_true, cls_pred=None, smooth=True):
    assert len(images) == len(cls_true)

    # Create figure with sub-plots.
    fig, axes = plt.subplots(3, 3)

    # Adjust vertical spacing.
    if cls_pred is None:
        hspace = 0.3
    else:
        hspace = 0.6
    fig.subplots_adjust(hspace=hspace, wspace=0.3)

    # Interpolation type.
    if smooth:
        interpolation = 'spline16'
    else:
        interpolation = 'nearest'

    for i, ax in enumerate(axes.flat):
        # There may be less than 9 images, ensure it doesn't crash.
        if i < len(images):
            # Plot image.
            ax.imshow(images[i],
                      interpolation=interpolation)

            # Name of the true class.
            cls_true_name = class_names[cls_true[i]]

            # Show true and predicted classes.
            if cls_pred is None:
                xlabel = "True: {0}".format(cls_true_name)
            else:
                # Name of the predicted class.
                cls_pred_name = class_names[cls_pred[i]]
                xlabel = "True: {0}\nPred: {1}".format(cls_true_name, cls_pred_name)
            # Show the classes as the label on the x-axis.
            ax.set_xlabel(xlabel)
        # Remove ticks from the plot.
        ax.set_xticks([])
        ax.set_yticks([])

    # Ensure the plot is shown correctly with multiple plots
    # in a single Notebook cell.
    plt.show()

def load_images(image_paths):
    # Load the images from disk.
    images = [plt.imread(path) for path in image_paths]
    # Convert to a numpy array and return it.
    return np.asarray(images)

In [ ]:
# Load the first images from the train-set.
images = load_images(image_paths=image_paths_train[0:9])
# Get the true classes for those images.
cls_true = cls_train[0:9]
# Plot the images and labels using our helper-function above.
plot_images(images=images, cls_true=cls_true, smooth=True)

# P3

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout

# Définition d'un modèle CNN simple
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(100, 100, 3)),
    MaxPooling2D(2, 2),

    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D(2, 2),

    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(1, activation='sigmoid')  # Sortie binaire
])

# Compilation du modèle
model.compile(optimizer=Adam(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])

# Résumé du modèle
model.summary()


# P4

In [ ]:
# model.add(Dense(1, activation='sigmoid'))
# model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])


In [ ]:
from keras.callbacks import EarlyStopping, ReduceLROnPlateau


# Callbacks

start = time.time()

import matplotlib.pyplot as plt

history = model.fit(train_batches, validation_data=valid_batches, epochs=100, verbose=1)

# Tracer les courbes d'entraînement et de validation
plt.plot(history.history['accuracy'], label='Training Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.legend()
plt.show()

plt.plot(history.history['loss'], label='Training Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.legend()
plt.show()


end = time.time()

print(f"Le modèle a été entraîné en {end - start:.2f} secondes.")

In [ ]:
end = time.time()

In [ ]:
print ("Model took %0.2f seconds to train"%(end - start))
result = model.evaluate(valid_batches, steps=1)
print("Test-set classification accuracy: {0:.2%}".format(result[1]))

# P5

In [ ]:
def plot_model_history(model_history):
    fig, axs = plt.subplots(1,2,figsize=(15,5))
    # summarize history for accuracy
    axs[0].plot(range(1,len(model_history.history['accuracy'])+1),model_history.history['accuracy'])
    axs[0].plot(range(1,len(model_history.history['val_accuracy'])+1),model_history.history['val_accuracy'])
    axs[0].set_title('Model Accuracy')
    axs[0].set_ylabel('Accuracy')
    axs[0].set_xlabel('Epoch')
    #axs[0].set_xticks(np.arange(1,len(model_history.history['accuracy'])+1),len(model_history.history['accuracy'])/10)
    axs[0].legend(['train', 'val'], loc='best')
    # summarize history for loss
    axs[1].plot(range(1,len(model_history.history['loss'])+1),model_history.history['loss'])
    axs[1].plot(range(1,len(model_history.history['val_loss'])+1),model_history.history['val_loss'])
    axs[1].set_title('Model Loss')
    axs[1].set_ylabel('Loss')
    axs[1].set_xlabel('Epoch')
    #axs[1].set_xticks(np.arange(1,len(model_history.history['loss'])+1),len(model_history.history['loss'])/10)
    axs[1].legend(['train', 'val'], loc='best')
    plt.show()

plot_model_history(history)

# P6

In [ ]:
import cv2
import matplotlib.pyplot as plt
import numpy as np
import glob  # Pour récupérer plusieurs fichiers à partir de dossiers

# Définir les chemins des images
car_images = glob.glob('data/valid/car/*')
not_car_images = glob.glob('data/valid/not_car/*')

# Sélectionner 10 images aléatoires de chaque classe
np.random.seed(42)  # Fixer la seed pour des tests reproductibles
selected_car_images = np.random.choice(car_images, 10, replace=False)
selected_not_car_images = np.random.choice(not_car_images, 10, replace=False)

# Combiner les chemins pour itérer
test_images = list(selected_car_images) + list(selected_not_car_images)
labels = ['car'] * 10 + ['not_car'] * 10  # Etiquettes pour les résultats attendus

# Fonction pour prédire et afficher les résultats
def predict_and_display(images, labels):
    for i, (img_path, label) in enumerate(zip(images, labels)):
        # Lire et préparer l'image
        img = cv2.imread(img_path)
        img_resized = cv2.resize(img, (100, 100))
        img_normalized = img_resized / 255.0  # Normalisation
        img_input = np.reshape(img_normalized, [1, 100, 100, 3])  # Ajouter la dimension batch

        # Prédire avec le modèle
        predic = model.predict(img_input)

        # Afficher l'image
        plt.figure(figsize=(3, 3))
        plt.imshow(cv2.cvtColor(img, cv2.COLOR_BGR2RGB))  # Convertir de BGR à RGB pour matplotlib
        plt.xticks([]), plt.yticks([])  # Supprimer les ticks de l'axe
        plt.title(f"Vraie classe : {label}")
        plt.show()

        # Afficher le résultat de la prédiction
        if predic[0][0] > 0.5:
            print(f"Prédiction : Ce n'est pas une voiture avec {predic[0][0] * 100:.2f}% de confiance")
        else:
            print(f"Prédiction : C'est une voiture avec {(1 - predic[0][0]) * 100:.2f}% de confiance")

# Tester le modèle sur les images sélectionnées
predict_and_display(test_images, labels)
